<a href="https://colab.research.google.com/github/mth-modelling/Assignment1-BikeShare/blob/main/AssignmentOne_Bikeshare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("hello friends :) - K")
print("hi guysssss - L")

import numpy as np
import matplotlib.pyplot as plt


hello friends :) - K
hi guysssss - L


What i think we should do, is separate this notebook into a few different sections, so section 1 will be our functions that we will initialise first, then we can have our main piece of code running the bikeshare system, and lastly our visualisation / plotting system


**Some ideas:**
- Maybe normalise all the values of 1) probability start rate and 2) probablity of a trip ending; maybe that will help us get values normalised for the entire system of bike stations
- Kacper smells


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


class Bikes:
    def __init__(self, station_ids, initial_bikes=10, capacity=16):
        self.station_ids = list(station_ids)
        self.bikes = {sid: initial_bikes for sid in self.station_ids}
        self.capacity = capacity
        #for capacity
        # capacity = dict(zip(df["station_id"],df["capacity"]))

    def move(self, start, end):
        if self.bikes[start] <= 0:
            return False
        if self.bikes[end] >= self.capacity:
            return False
        self.bikes[start] -= 1
        self.bikes[end] += 1
        return True


def normalise(w):
    s = sum(w.values())
    if s <= 0:
        raise ValueError("Weights must sum to > 0")
    return {k: v / s for k, v in w.items()}


def choose_weighted(rng, weight_dict):
    keys = list(weight_dict.keys())
    probs = np.array([weight_dict[k] for k in keys], dtype=float)
    probs = probs / probs.sum()
    return rng.choice(keys, p=probs)


def simulate_day(
    system,
    rng,
    start_w,
    end_w,
    T=960,            # 6am-10pm
    selection_number=4, #number of "customers" per each minute.
    allow_same_station_end=True,
    max_end_retries=10_000
):
    #annoyance counters
    annoy_total = 0
    annoy_no_bike = 0
    annoy_full_dock = 0


    annoy_no_bike_by_station = {sid: 0 for sid in system.station_ids}
    annoy_full_dock_by_station = {sid: 0 for sid in system.station_ids}


    bike_history = []
    annoy_history = []

    for t in range(T):  # each t is a minute
        for _ in range(selection_number):  # each iteration is a "customer"

            # 1) Customer chooses a start station by weights (do NOT filter empties)
            start = choose_weighted(rng, start_w)

            # If no bikes there, customer gives up; annoyance +1
            if system.bikes[start] <= 0:
                annoy_total += 1
                annoy_no_bike += 1
                annoy_no_bike_by_station[start] += 1
                continue  # next customer

            # 2) Customer has a bike; now must find an end station with free dock
            retries = 0
            while True:
                end = choose_weighted(rng, end_w)

                if not allow_same_station_end and end == start:
                    retries += 1
                    if retries > max_end_retries:
                        raise RuntimeError("Too many end retries (same-station constraint).")
                    continue

                if system.bikes[end] < system.capacity:
                    # success: perform the move
                    system.bikes[start] -= 1
                    system.bikes[end] += 1
                    break

                # end station is full -> annoyance +2, retry
                annoy_total += 2
                annoy_full_dock += 2
                annoy_full_dock_by_station[end] += 2

                retries += 1
                if retries > max_end_retries:
                    raise RuntimeError("Too many end retries: network may be saturated (all full?).")

        # log once per minute (after all customers that minute)
        bike_history.append({"t": t, **system.bikes})
        annoy_history.append({
            "t": t,
            "annoy_total": annoy_total,
            "annoy_no_bike": annoy_no_bike,
            "annoy_full_dock": annoy_full_dock
        })

    results = {
        "annoy_total": annoy_total,
        "annoy_no_bike": annoy_no_bike,
        "annoy_full_dock": annoy_full_dock,
        "annoy_no_bike_by_station": annoy_no_bike_by_station,
        "annoy_full_dock_by_station": annoy_full_dock_by_station,
        "final_bikes": system.bikes.copy(),
        # histories for plotting
        "bike_history": bike_history,
        "annoy_history": annoy_history,
    }
    return results


##running the simulation
rng = np.random.default_rng(1)  # setup seed for doing it again

station_ids = [1, 2, 3, 4]

    # initialise system: 4 stations, 5 bikes each, capacity 8
system = Bikes(
        station_ids=station_ids,
        initial_bikes=30,
        capacity=80
)

    # your weights (unchanged)
start_weights = normalise({
        1: 0.19,
        2: 0.35,
        3: 0.24,
        4: 0.35
})

end_weights = normalise({
        1: 0.10,
        2: 0.21,
        3: 0.03,
        4: 0.34
})


results = simulate_day(
        system=system,
        rng=rng,
        start_w=start_weights,
        end_w=end_weights,
        T=960,              #960 mins in one day for 18hrs (6-22)
        selection_number=1,
        allow_same_station_end=False
)

    #results
print("\nFinal bikes per station:")
for sid, count in results["final_bikes"].items():
    print(f"Station {sid}: {count}")

print("\nAnnoyance summary:")
print("Total annoyance:", results["annoy_total"])
print("No-bike annoyance:", results["annoy_no_bike"])
print("Full-dock annoyance:", results["annoy_full_dock"])

print("\nAnnoyance by station (no bikes):")
for sid, v in results["annoy_no_bike_by_station"].items():
        print(f"Station {sid}: {v}")

print("\nAnnoyance by station (full docks):")
for sid, v in results["annoy_full_dock_by_station"].items():
    print(f"Station {sid}: {v}")






bike_df = pd.DataFrame(results["bike_history"])
annoy_df = pd.DataFrame(results["annoy_history"])

# bikes over time per station
plt.figure()
for sid in station_ids:
    plt.plot(bike_df["t"], bike_df[sid], label=f"Station {sid}")
plt.xlabel("t (minutes)")
plt.ylabel("Bikes at station")
plt.legend()
plt.show()

    # annoyance totals over time
plt.figure()
plt.plot(annoy_df["t"], annoy_df["annoy_total"], label="Total annoyance")
plt.plot(annoy_df["t"], annoy_df["annoy_no_bike"], label="No-bike (+1)")
plt.plot(annoy_df["t"], annoy_df["annoy_full_dock"], label="Full-dock (+2)")
plt.xlabel("t (minutes)")
plt.ylabel("Annoyance")
plt.legend()
plt.show()

{1: 0.234, 2: 0.678, 3: 0.891, 4: 0.0323, 5: 0.0009}
{1: 0.127, 2: 0.369, 3: 0.485, 4: 0.018, 5: 0.0}


In [ ]:
import pandas as pd

# import excel spreadsheets

df = pd.read_excel('BelfastBikeApr21.xlsx')

df.columns = [c.replace(' ', '_') for c in df.columns]
df.columns = [c.replace('(hr^(-1))', '') for c in df.columns]
df.columns = [c.replace('(trip_end)', 'end') for c in df.columns]

start_rate = df.Start_rate_.to_numpy()
end_prob = df.p_end.to_numpy()
station_id = df.Station_id.to_numpy()